In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
no_exp_choice = [
    """//*[@id="sp_main_wrapper"]/div[1]/div[1]""",
    """//*[@id="sp_main_wrapper"]/div[1]/div[1]/div/div[1]/div[1]/label""",
    """//*[@id="sp_main_wrapper"]/div[1]/div[1]/div/div[3]/button[2]"""]
city_choice = [
    """//*[@id="sp_main_wrapper"]/div[2]/ul/li[2]/button/span[1]""",
    """//*[@id="depth1_btn_101000"]/button/span[1]""",
    """//*[@id="depth1_btn_102000"]/button/span[1]"""]
job_choice = [
    """//*[@id="sp_main_wrapper"]/div[2]/ul/li[1]/button/span[1]"""]

In [3]:
def option_click(xpath_list):
    for xpath in xpath_list:
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(1)

In [5]:
driver = webdriver.Chrome(service=Service('driver/chromedriver'))
driver.get("https://www.saramin.co.kr/zf_user/jobs/list/job-category")

In [6]:
choice_list = [no_exp_choice, city_choice, job_choice]
for choice in choice_list:
    option_click(choice)
    time.sleep(1)

In [7]:
# 검색창에 python 입력
skill_name = 'python'
element = driver.find_element_by_id("job_category_ipt_keyword")
element.send_keys(skill_name)
time.sleep(1)
xpath = """//*[@id="autocomplete_has_result"]/div[1]/div[2]/div/ul/li[1]/div/label"""
driver.find_element(By.XPATH, xpath).click()

C:\Users\dksqu\AppData\Local\Temp\ipykernel_20464\3849313762.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("job_category_ipt_keyword")


In [8]:
xpath = """//*[@id="search_btn"]"""
driver.find_element(By.XPATH, xpath).click()

### 데이터 수집

In [9]:
soup = BeautifulSoup(driver.page_source, 'lxml')

#### 기업 이름, 링크

In [10]:
company_info_soup = soup.select('div.common_recruilt_list div.col.company_nm a')

company_name = []
company_link = []

for company in company_info_soup:
    company_name.append(company.get('title'))
    company_link.append(company.get('href'))

#### 공고 제목

In [11]:
title_soup = soup.select('div.common_recruilt_list div.col.notification_info a.str_tit')

title = [title.get_text() for title in title_soup]

#### 지원자격

In [12]:
education_soup = soup.select('div.common_recruilt_list div.col.recruit_condition p.education')
education = [ condition.get_text() for condition in education_soup]

#### 지원 마감일

In [13]:
date_soup = soup.select('div.common_recruilt_list p.deadlines')
dates = [element.get_text() for element in date_soup]

In [14]:
date_list = [date.split('(') for date in dates]

In [15]:
last_date = []

for i in range(len(date_list)):
    if len(date_list[i]) == 3:
        date_str = '('.join(date_list[i][:2])
    else:
        date_str = date_list[i][0]
    
    last_date.append(date_str)

In [16]:
work_place= soup.select('div.common_recruilt_list work_place')
dates = [element.get_text() for element in date_soup]

In [17]:
data = {'기업이름':company_name, '공고제목':title, '지원자격':education, 
        '지원마감일':last_date, '링크':company_link}
df = pd.DataFrame(data)
df.head()

,기업이름,공고제목,지원자격,지원마감일,링크
0,(주)스쿨버스,(주)스쿨버스 모바일 클라이언트(플로터) 개발자 채용,학력무관,~ 04/19(화),/zf_user/jobs/relay/view?view_type=list&rec_id...
1,펑션베이㈜,CAE 소프트웨어 개발 / 엔지니어 모집,대학교(4년)↑,~ 04/17(일),/zf_user/jobs/relay/view?view_type=list&rec_id...
2,인터리젠(주),2022년 상반기 각 부문 인재 모집,"대학(2,3년)↑",~ 04/17(일),/zf_user/jobs/relay/view?view_type=list&rec_id...
3,(주)스칼라웍스,Data Scientist 채용 (병역특례 가능),학력무관,~ 05/30(월),/zf_user/jobs/relay/view?view_type=list&rec_id...
4,(주)오픈메이트,[오픈메이트] 플랫폼사업본부 신입/경력 인재모집,학력무관,~ 04/15(금),/zf_user/jobs/relay/view?view_type=list&rec_id...


In [18]:
df.tail()

,기업이름,공고제목,지원자격,지원마감일,링크
45,제트핑거소프트,웹프로그래머 채용 공고,"대학(2,3년)↑",~ 05/01(일),/zf_user/jobs/relay/view?view_type=list&rec_id...
46,국립암센터,국립암센터 인공지능사업팀 국가R&D사업 연구원 모집(신입/경력),학력무관,~ 04/15(금),/zf_user/jobs/relay/view?view_type=list&rec_id...
47,(주)컴트루테크놀로지,[AI인공지능] 딥러닝 아키텍쳐 개발 부문,대학교(4년)↑,~ 04/10(일),/zf_user/jobs/relay/view?view_type=list&rec_id...
48,(주)컴트루테크놀로지,"[인공지능 정보보안] 개발자 부문 - Web, Spring, C/C++",대학교(4년)↑,~ 04/10(일),/zf_user/jobs/relay/view?view_type=list&rec_id...
49,(주)윅슨어소시에이츠,"웹개발(PHP/백엔드) 1명 추가 채용,전원 재택근무","대학(2,3년)↑",~ 04/20(수),/zf_user/jobs/relay/view?view_type=list&rec_id...


In [19]:
work_place = soup.select('div.common_recruilt_list p.work_place')
dates = [element.get_text() for element in work_place]

In [20]:
print(len(work_place))

50


In [21]:
education = [element.get_text() for element in work_place]
print(education)

['서울 마포구', '경기 성남시 분당구', '서울 강남구', '서울 금천구', '서울 서대문구', '서울 강남구', '서울 강남구', '경기 고양시 덕양구', '서울 성동구', '서울전체', '서울전체', '서울 양천구', '경기 성남시 분당구', '경기 안산시 단원구', '서울 강남구', '서울 금천구', '서울 금천구', '서울 금천구', '서울 금천구', '서울 금천구', '경기 성남시 분당구', '서울 영등포구', '서울 마포구', '서울 강남구', '경기 성남시 분당구', '경기 안양시 동안구', '경기 고양시 일산서구', '서울 용산구', '서울 서초구', '서울 영등포구', '경기 성남시 분당구', '서울전체', '서울 금천구', '서울 강남구', '서울전체', '경기 성남시 분당구', '경기 성남시 분당구', '서울 영등포구', '서울 강남구', '서울 강서구', '서울 구로구', '경기 성남시 수정구', '경기 하남시', '서울 강남구', '서울 중구', '인천 부평구', '경기 고양시 일산동구', '서울 마포구', '서울 마포구', '서울 마포구']


In [22]:
data = {'기업이름':company_name, '공고제목':title, '지원자격':education, 
        '지원마감일':last_date, '링크':company_link,
        '주소': work_place}
    
df = pd.DataFrame(data)
df.head()

,기업이름,공고제목,지원자격,지원마감일,링크,주소
0,(주)스쿨버스,(주)스쿨버스 모바일 클라이언트(플로터) 개발자 채용,서울 마포구,~ 04/19(화),/zf_user/jobs/relay/view?view_type=list&rec_id...,[서울 마포구]
1,펑션베이㈜,CAE 소프트웨어 개발 / 엔지니어 모집,경기 성남시 분당구,~ 04/17(일),/zf_user/jobs/relay/view?view_type=list&rec_id...,[경기 성남시 분당구]
2,인터리젠(주),2022년 상반기 각 부문 인재 모집,서울 강남구,~ 04/17(일),/zf_user/jobs/relay/view?view_type=list&rec_id...,[서울 강남구]
3,(주)스칼라웍스,Data Scientist 채용 (병역특례 가능),서울 금천구,~ 05/30(월),/zf_user/jobs/relay/view?view_type=list&rec_id...,[서울 금천구]
4,(주)오픈메이트,[오픈메이트] 플랫폼사업본부 신입/경력 인재모집,서울 서대문구,~ 04/15(금),/zf_user/jobs/relay/view?view_type=list&rec_id...,[서울 서대문구]


In [23]:
df.tail()

,기업이름,공고제목,지원자격,지원마감일,링크,주소
45,제트핑거소프트,웹프로그래머 채용 공고,인천 부평구,~ 05/01(일),/zf_user/jobs/relay/view?view_type=list&rec_id...,[인천 부평구]
46,국립암센터,국립암센터 인공지능사업팀 국가R&D사업 연구원 모집(신입/경력),경기 고양시 일산동구,~ 04/15(금),/zf_user/jobs/relay/view?view_type=list&rec_id...,[경기 고양시 일산동구]
47,(주)컴트루테크놀로지,[AI인공지능] 딥러닝 아키텍쳐 개발 부문,서울 마포구,~ 04/10(일),/zf_user/jobs/relay/view?view_type=list&rec_id...,[서울 마포구]
48,(주)컴트루테크놀로지,"[인공지능 정보보안] 개발자 부문 - Web, Spring, C/C++",서울 마포구,~ 04/10(일),/zf_user/jobs/relay/view?view_type=list&rec_id...,[서울 마포구]
49,(주)윅슨어소시에이츠,"웹개발(PHP/백엔드) 1명 추가 채용,전원 재택근무",서울 마포구,~ 04/20(수),/zf_user/jobs/relay/view?view_type=list&rec_id...,[서울 마포구]


In [24]:
for i, address in enumerate(work_place):
    button = driver.find_element_by_work_place('a')
    if len(work_place[i]) == '서울전체': 
    
    if len(work_place[i]) == '경기 전체':
        
    print(work_place)
    
    
    


[<p class="work_place">서울 마포구</p>, <p class="work_place">경기 성남시 분당구</p>, <p class="work_place">서울 강남구</p>, <p class="work_place">서울 금천구</p>, <p class="work_place">서울 서대문구</p>, <p class="work_place">서울 강남구</p>, <p class="work_place">서울 강남구</p>, <p class="work_place">경기 고양시 덕양구</p>, <p class="work_place">서울 성동구</p>, <p class="work_place">서울전체</p>, <p class="work_place">서울전체</p>, <p class="work_place">서울 양천구</p>, <p class="work_place">경기 성남시 분당구</p>, <p class="work_place">경기 안산시 단원구</p>, <p class="work_place">서울 강남구</p>, <p class="work_place">서울 금천구</p>, <p class="work_place">서울 금천구</p>, <p class="work_place">서울 금천구</p>, <p class="work_place">서울 금천구</p>, <p class="work_place">서울 금천구</p>, <p class="work_place">경기 성남시 분당구</p>, <p class="work_place">서울 영등포구</p>, <p class="work_place">서울 마포구</p>, <p class="work_place">서울 강남구</p>, <p class="work_place">경기 성남시 분당구</p>, <p class="work_place">경기 안양시 동안구</p>, <p class="work_place">경기 고양시 일산서구</p>, <p class="work_place">서울 용산구</p>, <p class="work_p